In [20]:
# Cell 1: Setup logging with timestamps and durations
import logging
import os
import time
from datetime import datetime

class TimestampedLogger:
    def __init__(self, log_dir='logs', log_file='simulation.log'):
        os.makedirs(log_dir, exist_ok=True)
        self.log_path = os.path.join(log_dir, log_file)
        logging.basicConfig(filename=self.log_path, level=logging.INFO, filemode='w')
        self.start_time = time.time()
        self.last_time = self.start_time
        self.log("Logger initialized.")

    def _now(self):
        return datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    def _duration(self):
        current_time = time.time()
        duration = current_time - self.last_time
        self.last_time = current_time
        return f"{duration:.3f}s"

    def log(self, message):
        timestamp = self._now()
        duration = self._duration()
        log_message = f"[{timestamp}] (+{duration}) {message}"
        print(log_message)
        logging.info(log_message)

logger = TimestampedLogger()


[2025-06-30 01:52:14] (+0.000s) Logger initialized.


In [21]:
# Cell 2: Define data structures for the simulation environment
from dataclasses import dataclass, field
from typing import Tuple, List, Optional
import random
import uuid

@dataclass
class Tile:
    x: int
    y: int
    piece: Optional[str] = None  # e.g. "King", "Pawn", etc.
    drones: List[str] = field(default_factory=list)  # e.g. ["Drone_1", "Drone_4"]

@dataclass
class Drone:
    id: int
    position: Tuple[int, int]
    memory: dict = field(default_factory=dict)

@dataclass
class King:
    """Represents the king on the board."""
    position: Tuple[int, int]

@dataclass
class SimulationState:
    """Tracks the state of the simulation."""
    grid_size: Tuple[int, int]
    tiles: List[List[Tile]]
    drones: List[Drone]
    king: King
    turn: int = 0


In [22]:
# Cell 3: Initialize the simulation grid and entities
def initialize_simulation(grid_size: Tuple[int, int], num_drones: int) -> SimulationState:
    """Initializes the grid, drones, and king."""
    width, height = grid_size
    tiles = [[Tile(x=x, y=y) for y in range(height)] for x in range(width)]

    # Place king at the center
    king_pos = (width // 2, height // 2)
    king = King(position=king_pos)
    tiles[king_pos[0]][king_pos[1]].piece = 'King'

    # Place drones randomly, avoiding the King's tile
    drones = []
    for i in range(num_drones):
        pos = [king_pos[0], king_pos[1]]
        drone = Drone(
            id=i,
            position=pos
        )
        tiles[pos[0]][pos[1]].drones.append(f"Drone_{i}")
        drones.append(drone)

    return SimulationState(
        grid_size=grid_size,
        tiles=tiles,
        drones=drones,
        king=king,
        turn=0
    )


In [23]:
# Cell 4: Load configuration from a JSON file
import json

def load_config(config_path: str = "config.json") -> dict:
    """Loads the configuration file."""
    with open(config_path, "r") as f:
        return json.load(f)


In [24]:
# Cell 5: Define the simulation runner class
# class Simulation:
#     def __init__(self, config, state: SimulationState, logger: TimestampedLogger):
#         self.config = config
#         self.state = state
#         self.logger = logger

#     def run(self):
#         self.logger.log("Starting simulation.")
#         max_turns = self.config["simulation"].get("max_turns", 10)

#         for turn in range(max_turns):
#             self.state.turn = turn
#             self.logger.log(f"Turn {turn + 1} started.")

#             # Each drone could act here
#             for drone in self.state.drones:
#                 self.logger.log(f"{drone.id} is at {drone.position}.")

#             self.logger.log(f"Turn {turn + 1} ended.")
#         self.logger.log("Simulation finished.")


In [25]:
# Cell 6: Load config and initialize the simulation
config = load_config("config.json")
logger.log("Config loaded.")

state = initialize_simulation(
    grid_size=tuple(config["board"]["size"]),
    num_drones=config["simulation"]["num_drones"]
)
logger.log("Simulation initialized.")


[2025-06-30 01:52:14] (+0.099s) Config loaded.
[2025-06-30 01:52:14] (+0.000s) Simulation initialized.


In [26]:
# Cell 7: Define and run the simulation loop
# def run_simulation(state: SimulationState, max_turns: int = 5):
#     """Runs the simulation for a given number of turns."""
#     for turn in range(max_turns):
#         logger.log(f"--- Turn {turn + 1} ---")
#         # Log drone positions
#         for drone in state.drones:
#             logger.log(f"{drone.id} is at {drone.position}")
#         state.turn += 1
#         time.sleep(0.5)

# run_simulation(state, max_turns=5)
# logger.log("Simulation complete.")


In [27]:
# Cell 8: Function to draw the board using pygame

# def draw_board(screen, state, config):
#     import pygame

#     screen.fill((255, 255, 255))
#     grid_width, grid_height = state.grid_size
#     tile_size = config["gui"]["tile_size"]
#     margin = config["gui"]["margin"]
#     font = pygame.font.SysFont(None, 24)

#     for x in range(grid_width):
#         for y in range(grid_height):
#             tile = state.tiles[x][y]
#             rect = pygame.Rect(
#                 x * tile_size + margin,
#                 y * tile_size + margin,
#                 tile_size - margin,
#                 tile_size - margin
#             )
#             pygame.draw.rect(screen, (200, 200, 200), rect)

#             if tile.piece == "King":
#                 pygame.draw.rect(screen, (255, 215, 0), rect)
#                 text = font.render("K", True, (0, 0, 0))
#                 screen.blit(text, rect.topleft)
            
#             if tile.drones:
#                 pygame.draw.circle(screen, (100, 100, 255), rect.center, tile_size // 4)
#                 ids = ','.join(str(d) for d in tile.drones)
#                 text = font.render(ids, True, (0, 0, 0))
#                 screen.blit(text, (rect.left + 2, rect.top + 2))


In [28]:
# Cell 9: Run the simulation logic step by step
# def run_simulation(state: SimulationState, steps: int = 10):
#     """Runs the simulation for a given number of steps."""
#     logger.log(f"Starting simulation for {steps} steps.")
    
#     for step in range(steps):
#         logger.log(f"Step {step+1} started.")

#         # Placeholder for real drone logic (movement, communication, etc.)
#         for drone in state.drones:
#             # Example: log drone ID and position
#             logger.log(f"Drone {drone.id} at position {drone.position}")

#         state.turn += 1
#         logger.log(f"Step {step+1} finished.")
    
#     logger.log("Simulation completed.")


In [29]:
# Cell 10: Main entry point to execute the simulation
# if __name__ == "__main__" or True:  # `or True` allows execution in Jupyter
#     logger.log("Launching simulation...")
#     run_simulation(state, steps=config["simulation"]["steps"])


In [ ]:
# Cell 11: Define real-time simulation GUI using Pygame

import pygame

def draw_field(screen, state, config):
    """Draws the board, king, and drones using the current state."""
    cell_size = config["gui"]["cell_size"]
    margin = config["gui"]["margin"]
    grid_width, grid_height = state.grid_size

    screen.fill(config["gui"]["background_color"])
    font = pygame.font.SysFont(None, 16)

    for x in range(grid_width):
        for y in range(grid_height):
            rect = pygame.Rect(
                x * (cell_size + margin) + margin,
                y * (cell_size + margin) + margin,
                cell_size,
                cell_size
            )
            pygame.draw.rect(screen, config["gui"]["grid_color"], rect)

            tile = state.tiles[x][y]

            # Draw King
            if tile.piece == "King":
                pygame.draw.circle(screen, config["gui"]["king_color"], rect.center, cell_size // 3)

            # Draw drones (multiple per tile supported)
            drone_ids = tile.drones
            total = len(drone_ids)
            if total > 0:
                angle_step = 360 / total if total > 1 else 0
                radius = cell_size // 6

                for idx, drone_id in enumerate(drone_ids):
                    print(f"Drawing drone {drone_id} at tile ({x}, {y})")
                    angle = angle_step * idx
                    offset = pygame.math.Vector2(0, 0)
                    if total > 1:
                        offset = pygame.math.Vector2(1, 0).rotate(angle) * (cell_size // 4)
                    center = (rect.centerx + int(offset.x), rect.centery + int(offset.y))

                    pygame.draw.circle(screen, config["gui"]["drone_color"], center, radius)
                    text = font.render(str(idx + 1), True, config["gui"]["text_color"])
                    text_rect = text.get_rect(center=center)
                    screen.blit(text, text_rect)

def run_simulation_with_gui(state: SimulationState, config: dict, logger: TimestampedLogger):
    """Runs the simulation with real-time GUI display using Pygame."""

    pygame.init()

    grid_width, grid_height = state.grid_size
    cell_size = config["gui"]["cell_size"]
    margin = config["gui"]["margin"]
    max_turns = config["simulation"].get("max_turns", 10)
    delay = config["simulation"].get("delay", 300)

    window_width = grid_width * (cell_size + margin) + margin
    window_height = grid_height * (cell_size + margin) + margin
    screen = pygame.display.set_mode((window_width, window_height))
    pygame.display.set_caption("Real-Time Drone Simulation")

    directions = [(-1, -1), (-1, 0), (-1, 1),
                  ( 0, -1), ( 0, 0), ( 0, 1),
                  ( 1, -1), ( 1, 0), ( 1, 1)]

    running = True
    clock = pygame.time.Clock()

    for step in range(max_turns):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
                break
        if not running:
            break

        # Move each drone
        for drone in state.drones:
            drone_id = f'Drone_{drone.id}'
            x, y = drone.position
            old_tile = state.tiles[x][y]

            # Remove drone from current tile before movement
            if drone_id in old_tile.drones:
                old_tile.drones.remove(drone_id)

            # Determine valid moves
            valid_moves = []
            for dx, dy in directions:
                nx, ny = x + dx, y + dy
                if 0 <= nx < grid_width and 0 <= ny < grid_height:
                    valid_moves.append((nx, ny))

            # Move drone
            if valid_moves:
                new_pos = random.choice(valid_moves)
                drone.position = new_pos
                new_tile = state.tiles[new_pos[0]][new_pos[1]]
                new_tile.drones.append(drone_id)
            else:
                # If no valid moves, re-add to current tile
                old_tile.drones.append(drone_id)


        state.turn += 1
        logger.log(f"Turn {state.turn} completed.")

        draw_field(screen, state, config)

        pygame.display.flip()
        pygame.time.delay(delay)
        clock.tick(30)

    logger.log("Simulation ended.")
    pygame.quit()


In [31]:
# Cell 12: Optional launcher for real-time simulation with GUI in notebooks

RUN_SIMULATION = True  # Set to False to disable auto-execution in notebooks

if __name__ == "__main__" or RUN_SIMULATION:
    logger.log("Launching real-time simulation with GUI...")
    run_simulation_with_gui(state, config, logger)


[2025-06-30 01:52:14] (+0.097s) Launching real-time simulation with GUI...
[2025-06-30 01:52:14] (+0.134s) Turn 1 completed.
Drawing drone Drone_2 at tile (4, 5)
Drawing drone Drone_0 at tile (4, 6)
Drawing drone Drone_1 at tile (5, 4)
Drawing drone Drone_0 at tile (5, 5)
Drawing drone Drone_1 at tile (5, 5)
Drawing drone Drone_2 at tile (5, 5)
Drawing drone Drone_3 at tile (5, 5)
Drawing drone Drone_4 at tile (5, 5)
Drawing drone Drone_3 at tile (6, 4)
Drawing drone Drone_4 at tile (6, 4)
[2025-06-30 01:52:15] (+0.303s) Turn 2 completed.
Drawing drone Drone_2 at tile (4, 5)
Drawing drone Drone_0 at tile (4, 5)
Drawing drone Drone_0 at tile (4, 6)
Drawing drone Drone_1 at tile (5, 4)
Drawing drone Drone_0 at tile (5, 5)
Drawing drone Drone_1 at tile (5, 5)
Drawing drone Drone_2 at tile (5, 5)
Drawing drone Drone_3 at tile (5, 5)
Drawing drone Drone_4 at tile (5, 5)
Drawing drone Drone_2 at tile (5, 5)
Drawing drone Drone_1 at tile (6, 3)
Drawing drone Drone_3 at tile (6, 4)
Drawing dro